# (제목변경) 자동화 코드 주제

### 디렉토리 설정
- 아래 각 변수에 해당하는 알맞은 경로를 찾아 입력해주세요.
- 경로 주소는 '.../.../...' 형식을 따라주세요.

In [ ]:
class directory:
    dropbox_dir = 'C:/user/MADUP/Dropbox (주식회사매드업)'
    # 드롭박스 폴더 위치경로
        
    download_dir = 'C:/Users/MADUP/Downloads'
    # 최종 결과파일을 다운받을 폴더 위치경로

dr = directory()

### 날짜 설정
- 보통 작일자 기준으로 요청주신 기간의 데이터를 출력하게 됩니다.
- 날짜 조정이 필요한 경우 아래 변수 값을 수정하여 출력해주세요.

In [ ]:
import datetime
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

class report_date:
    today = datetime.date.today()
    # 오늘 날짜 ex> 2022-09-14
    
    # today = datetime.date(year = 2022, month = 8, day = 12)
    # 다른 일자를 기준으로 추출하고 싶은 경우 위 코드 수정 및 주석해제
    
    day_1 = today - datetime.timedelta(1)
    # 전일자 ex> 2022-09-13
    
    start_day = datetime.date(year=day_1.year, month=day_1.month, day=1)
    # 금월 첫 1일자 ex> 2022-09-01
    
    yearmonth = day_1.strftime('%Y%m')
    # 년월 ex> 202209
    
    month_name = str(day_1.month) + '월'
    # 금월 ex> 9월

rdate = report_date()

### 구글 스프레드 시트 읽기
- 특정 구글 스프레드 시트를 읽어오기 위해 활용하는 메소드 모음입니다.
- dcteam@madup-355605.iam.gserviceaccount.com 계정에 참조하고자 하는 구글 시트 공유가 필요합니다.

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import time


class spreadsheet:
    token_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/token'
    
    def spread_document_read(self, url):
        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        json_file = token_dir + '/madup-355605-cd37b0ac201f.json'
        credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file, scope)
        gc = gspread.authorize(credentials)
        read = gc.open_by_url(url)
        print('스프레드시트 읽기 완료')
        return read
    
    def spread_sheet(self, doc, sheet_name, col_num=0, row_num=0):
        while True :
            try :
                data_sheet = doc.worksheet(sheet_name)
                data_sheet_read = data_sheet.get_all_values()
                result = pd.DataFrame(data_sheet_read, columns=data_sheet_read[row_num]).iloc[row_num+1:, col_num:]
            except :
                print('API 오류로 15초 후 다시 시도 합니다.')
                time.sleep(15)
                continue
            break
        return result

spreadsheet = spreadsheet()

### 코드
- 전처리 자동화를 위한 코드로 임의 수정을 하실 경우 복사본 생성하여 진행해주시길 바랍니다.
- 혹 아래 이미지와 같은 형식의 오류가 발생할 경우 `!pip install 패키지명`형식의 코드 작성하여 실행해주시면 해당 패키지가 설치됩니다.
    - <img src = 'https://ifh.cc/g/ymACFT.png' align='left'>

- 이외의 오류가 발생하거나 문제가 해결되지 않는 경우, 데컨팀에 문의주세요 :)

In [ ]:
# 패키지 설치 코드 (패키지 설치 오류 발생시 아래 코드 실행해주세요.)
#!pip install seleniumㄴ
#!pip install datetime
#!pip install warnings

참조시트: https://docs.google.com/spreadsheets/d/14i42NrpnA_9k8nCgyc4Y5KssLP8tOOXUyzYw0un7qXY/edit#gid=211027705
- 구글 스프레드 시트에 새로운 탭 광고주 명으로 생성한 후 url 리스트 작성
- TOTAL 시트에 광고주 명, 문장 패턴 검색 체크, 검색 문구 (여러개인 경우 '/'로 구분) 작성

위 작업을 모두 이행한 후 코드 실행해야 작업이 가능합니다.

In [ ]:
# 광고주 명 입력
ADVERTISER = '잡코리아'

In [ ]:
import pandas as pd
import numpy as np
from selenium import webdriver
from pytz import timezone
import datetime


doc = spreadsheet.spread_document_read("https://docs.google.com/spreadsheets/d/14i42NrpnA_9k8nCgyc4Y5KssLP8tOOXUyzYw0un7qXY/edit#gid=211027705")
result_dir = dr.download_dir


def get_url_check_list(doc, ADVERTISER):
    url_check_list = spreadsheet.spread_sheet(doc, ADVERTISER).reset_index(drop=True)
    url_check_list = url_check_list[['id', 'url']].replace('', np.nan).dropna().reset_index(drop=True)

    return url_check_list


def link_validation_check(url_check_list, checker):
    result_df = pd.DataFrame(columns=['id', 'url', 'checker'])
    for index, row in url_check_list.iterrows():
        driver = webdriver.Chrome(dr.dropbox_dir + '/광고사업부/데이터컨설팅/token/chromedriver')
        url = row['url']
        try:
            print(url)
            driver.get(url)
            driver.implicitly_wait(500)
            page_source = driver.page_source
            for val in checker:
                if val in page_source:
                    row['checker'] = val
                    result_df = result_df.append(row)
        except Exception as e:
            print(f'{e}\n url 문제 발생:', url)
        driver.quit()

    return result_df


def landing_expired_check(url_check_list):
    landing_expired_result_df = pd.DataFrame(columns=['id', 'url'])
    return landing_expired_result_df


def redirect_check(url_check_list):
    redirect_result_df = pd.DataFrame(columns=['id', 'url'])
    return redirect_result_df


def download_df(link_validation_result_df, landing_expired_result_df, redirect_result_df, result_dir):
    time = datetime.datetime.now(timezone("Asia/Seoul")).strftime("%y년_%m월_%d일_%H시")
    writer = pd.ExcelWriter(result_dir + f'/{ADVERTISER}_랜딩페이지_점검_{time}.xlsx', engine='xlsxwriter', engine_kwargs={'options': {'strings_to_urls': False}})
    link_validation_result_df.to_excel(writer, sheet_name = 'link_validation_check', index=False)
    # landing_expired_result_df.to_excel(writer, sheet_name='landing_expired_check', index=False)
    # redirect_result_df.to_excel(writer, sheet_name='redirect_check', index=False)
    writer.close()


def landing_check_solution_exec(doc, ADVERTISER):
    url_check_list = get_url_check_list(doc, ADVERTISER)

    total_index = spreadsheet.spread_sheet(doc, 'TOTAL')
    total_index = total_index.drop_duplicates('업체명', keep='last')
    index = total_index.loc[total_index['업체명'] == ADVERTISER].reset_index(drop=True)

    link_validation_result_df = pd.DataFrame(columns=['id', 'url'])
    landing_expired_result_df = pd.DataFrame(columns=['id', 'url'])
    redirect_result_df = pd.DataFrame(columns=['id', 'url'])
    if index['문장 패턴 검색'].values == 'TRUE':
        checker = index['검색 문구'][0].split('/')
        link_validation_result_df = link_validation_check(url_check_list, checker)
    elif index['접속 불가 점검(미구현)'].values == 'TRUE':
        landing_expired_result_df = landing_expired_check(url_check_list)
    elif index['리다이렉트 점검(미구현)'].values == 'TRUE':
        redirect_result_df = redirect_check(url_check_list)

    download_df(link_validation_result_df, landing_expired_result_df, redirect_result_df, result_dir)


landing_check_solution_exec(doc, ADVERTISER)